#### 1. Pending approach- learn phrases and then learn vectors, pass them in classifier and show frequent phrases from each pos and neg corpus.

In [1]:
import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
import numpy as np # high dimensional vector computing library.
from copy import deepcopy
from string import punctuation
from random import shuffle
import json
import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below
# from gensim.models.doc2vec import TaggedDocument

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

import nltk
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale
import pickle

import os
os.environ['MKL_NUM_THREADS'] = '12'
os.environ['GOTO_NUM_THREADS'] = '12'
os.environ['OMP_NUM_THREADS'] = '12'
os.environ['openmp'] = 'True'

from keras.models import model_from_json
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Using TensorFlow backend.


ImportError: cannot import name TweetTokenizer

In [ ]:
import re
from nltk import RegexpTokenizer
import nltk
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
import collections
import ast
from nltk.corpus import wordnet as wn

from datetime import datetime

from pattern.en import tokenize,parse

from nltk.corpus import wordnet

In [4]:
os.chdir("deeplearning/data")
directory = os.getcwd()
trainPath = r"/tweets_data/train.csv"
def ingest(path):
    data = pd.read_csv(directory + path)
    data.drop(['id', 'device',"datetime","user"], axis=1, inplace=True)
    data = data[data.sentiment.isnull() == False]
    data['sentiment'] = data['sentiment'].map(int)
    data['sentiment'] = data['sentiment'].map( {4:1, 0:0} )
    data = data[data['text'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    print ('dataset loaded with shape', data.shape)
    return data

data = ingest(trainPath)

('dataset loaded with shape', (1600000, 2))


In [6]:
def tokenize(tweet):
    try:
        tweet =  tweet.lower()
        tokens = tokenizer.tokenize(tweet)
        tokens = list(filter(lambda t: not t.startswith('@'), tokens))
        tokens = list(filter(lambda t: not t.startswith('#'), tokens))
        tokens = list(filter(lambda t: not t.startswith('http'), tokens))
        return tokens
    except:
        return 'NC'

In [7]:
def postprocess(data, n=data.shape[0]):
    data = data.head(n)
    data['tokens'] = data['text'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

data = postprocess(data)
# test_data = postprocess(test_data)

progress-bar: 100%|██████████| 1600000/1600000 [02:20<00:00, 11352.23it/s]


In [ ]:
# x_train, x_val, y_train, y_val = train_test_split(np.array(data.head(n).tokens),
#                                                 np.array(data.head(n).sentiment), test_size=0.2)

X_train, X_test, y_train, y_test = train_test_split(np.array(data.tokens), np.array(data.sentiment), test_size=0.1, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

In [ ]:
from scipy.stats import itemfreq

print("trainset labels distribution")
print(itemfreq(y_train))
print
print("valset labels distribution")
print (itemfreq(y_val))
print
print("testset labels distribution")
print( itemfreq(y_test))

In [ ]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

X_train = labelizeTweets(X_train, 'TRAIN')
X_val = labelizeTweets(X_val, 'Val')
X_test = labelizeTweets(X_test, 'TEST')

In [ ]:
n_dim = 200
tweet_w2v = Word2Vec(size=n_dim, min_count=10)
tweet_w2v.build_vocab([x.words for x in tqdm(X_train)])
tweet_w2v.train([x.words for x in tqdm(X_train)],total_examples=tweet_w2v.corpus_count, epochs=tweet_w2v.iter)

In [ ]:
tweet_w2v.save(directory + "/saved_models/equal_dist_Word2Vec_model_200dims")

### Load word2vec model

In [3]:
os.chdir("deeplearning/data")
directory = os.getcwd()
tweet_w2v = Word2Vec.load(directory + "/saved_models/equal_dist_Word2Vec_model_200dims")  # you can continue training with the loaded model!

In [4]:
tweet_w2v.most_similar('good')

[(u'goood', 0.7326517105102539),
 (u'great', 0.726836085319519),
 (u'gooood', 0.6671431064605713),
 (u'rough', 0.6489897966384888),
 (u'goooood', 0.6336358189582825),
 (u'fantastic', 0.6276979446411133),
 (u'tough', 0.6255412697792053),
 (u'gd', 0.615630030632019),
 (u'fab', 0.6139873266220093),
 (u'nice', 0.6114063262939453)]

In [5]:
tweet_w2v.most_similar('bar')

[(u'cafe', 0.6940361261367798),
 (u'pub', 0.6776139736175537),
 (u'lounge', 0.6597999334335327),
 (u'club', 0.6557016372680664),
 (u'restaurant', 0.6481664180755615),
 (u'table', 0.6430073976516724),
 (u'grill', 0.6275577545166016),
 (u'gate', 0.6055143475532532),
 (u'casino', 0.6018807888031006),
 (u'bakery', 0.5978732705116272)]

In [6]:
tweet_w2v.most_similar('facebook')

[(u'fb', 0.8944979310035706),
 (u'myspace', 0.8294822573661804),
 (u'bebo', 0.7485839128494263),
 (u'twitter', 0.7347208261489868),
 (u'linkedin', 0.7318819165229797),
 (u'flickr', 0.7263391017913818),
 (u'yahoo', 0.7208259105682373),
 (u'msn', 0.7146008014678955),
 (u'lj', 0.7127932906150818),
 (u'aim', 0.6888376474380493)]

### TF-IDF matrix for weighted average

In [14]:
print ('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

building tf-idf matrix ...


NameError: name 'X_train' is not defined

### save TF-IDF matrix

In [ ]:
# directory = r"/deeplearning/data"
# with open (directory+'/saved_models/equal_dist_tf-idf', 'wb') as fp:
#     pickle.dump(tfidf, fp, protocol=2)

### Load Saved tf-idf

In [7]:
# import cPickle as pickle
directory = r"deeplearning/data"
with open (directory+'/saved_models/equal_dist_tf-idf', 'rb') as fp:
    tfidf = pickle.load(fp)

In [8]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
n_dim = 200
train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, X_train))])
train_vecs_w2v = scale(train_vecs_w2v)

val_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, X_val))])
val_vecs_w2v = scale(val_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, X_test))])
test_vecs_w2v = scale(test_vecs_w2v)

### Get vectors for out of Sample Data

In [17]:
path = r"/tweets_data/amazon_text_for_sentiment.csv"
new_words = pd.read_csv(directory + path)
new_words.shape

/usr/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2741: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler)


(460023, 5)

In [ ]:
path = r"/tweets_data/amazon_text_for_sentiment.csv"
def ingest(path):
    data = pd.read_csv(directory + path)
    data.drop(['_id',"quoted_status_sentiment"], axis=1, inplace=True) # extra cols'tweet_text_sentiment',"tweet_quoted_status_text"
    data = data[data['tweet_text'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    print ('dataset loaded with shape', data.shape)
    return data

new_data = ingest(path)

In [ ]:
def tokenize(tweet):
    try:
        tweet =  tweet.lower()
        tokens = tokenizer.tokenize(tweet)
        tokens = list(filter(lambda t: not t.startswith('@'), tokens))
        tokens = list(filter(lambda t: not t.startswith('#'), tokens))
        tokens = list(filter(lambda t: not t.startswith('http'), tokens))
        return tokens
    except:
        return 'NC'

In [ ]:
def postprocess(data):
    data['tokens'] = data['tweet_text'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

new_data = postprocess(new_data)

In [ ]:
# new_data.to_csv(directory + "/tweets_data/cleaned_amazon_reviews.csv")
# new_data = pd.read_csv(directory + "/tweets_data/cleaned_amazon_reviews.csv")

In [ ]:
# new_data = new_data.drop(['Unnamed: 0'], axis=1)

In [ ]:
new_data.head()

In [ ]:
new_data.head()

In [ ]:
OOS_data = np.array(new_data.tokens)

In [ ]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

OOS_data = labelizeTweets(OOS_data, 'OOS')

In [ ]:
n_dim = 200
OOS_vectors = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, OOS_data))])
OOS_vectors = scale(OOS_vectors)

### Saving word vectors- don't run it again

In [ ]:
#OOS vectors
# directory = r"deeplearning/data"
# with open(directory+'/word2vec/equal_dist_OOS_wordvectors_v3', 'wb') as fp:
#     pickle.dump(OOS_vectors, fp, protocol=2)

In [ ]:
# import pickle
# directory = r"deeplearning/data"
# #train
# with open(directory+'/word2vec/equal_dist_train_wordvecs_v3', 'wb') as fp:
#     pickle.dump(train_vecs_w2v, fp, protocol=2)

# with open(directory+'/word2vec/equal_dist_trainw2v_label_v3', 'wb') as fp:
#     pickle.dump(y_train, fp, protocol=2)

# #val    
# with open(directory+'/word2vec/equal_dist_val_wordvecs_v3', 'wb') as fp:
#     pickle.dump(val_vecs_w2v, fp, protocol=2)
    
# with open(directory+'/word2vec/equal_dist_valw2v_label_v3', 'wb') as fp:
#     pickle.dump(y_val, fp, protocol=2)
    
# #test   
# with open(directory+'/word2vec/equal_dist_test_wordvecs_v3', 'wb') as fp:
#     pickle.dump(test_vecs_w2v, fp, protocol=2)
    
# with open(directory+'/word2vec/equal_dist_testw2v_label_v3', 'wb') as fp:
#     pickle.dump(y_test, fp, protocol=2)

### Load Vectors

In [9]:
directory = r"deeplearning/data"
with open (directory+'/word2vec/equal_dist_train_wordvecs_v3', 'rb') as fp:
    train_vectors = pickle.load(fp)

with open (directory+'/word2vec/equal_dist_trainw2v_label_v3', 'rb') as fp:
    train_label = pickle.load(fp)

with open (directory+'/word2vec/equal_dist_val_wordvecs_v3', 'rb') as fp:
    val_vectors = pickle.load(fp)

with open (directory+'/word2vec/equal_dist_valw2v_label_v3', 'rb') as fp:
    val_label = pickle.load(fp)
    
with open (directory+'/word2vec/equal_dist_test_wordvecs_v3', 'rb') as fp:
    test_vectors = pickle.load(fp)

with open (directory+'/word2vec/equal_dist_testw2v_label_v3', 'rb') as fp:
    test_label = pickle.load(fp)

#out of sample vectors
# with open (directory+'/word2vec/equal_dist_OOS_wordvectors_v3', 'rb') as fp:
#     OOS_vectors = pickle.load(fp)


### Logistic 

In [ ]:
from sklearn.linear_model import SGDClassifier

lr = SGDClassifier(loss='log', penalty='l1')
lr.fit(train_vectors, train_label)

print (lr.score(val_vectors, val_label))
print (lr.score(test_vectors, test_label))

In [ ]:
lr_prob = lr.predict_proba(OOS_vectors)

In [ ]:
df_new_colwise=pd.concat([new_data,pd.DataFrame(lr_prob)],axis=1)
df_new_colwise.sort_values(by=1, ascending=True).head(100)

### NN

In [ ]:
### Latest Model word2vec
from keras.optimizers import SGD, RMSprop, adam
from keras.models import Sequential
from keras.layers import Dense

import os
os.environ['MKL_NUM_THREADS'] = '20'
os.environ['GOTO_NUM_THREADS'] = '20'
os.environ['OMP_NUM_THREADS'] = '20'
os.environ['openmp'] = 'True'

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=200))
# model.add(Dense(16, activation='relu') )
# model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
opt = adam(lr=0.001)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_vectors, train_label, nb_epoch=10, batch_size=32, verbose=1)

In [ ]:
score = model.evaluate(val_vectors, val_label, batch_size=128, verbose=2)
print score[1]

In [ ]:
score = model.evaluate(test_vectors, test_label, batch_size=128, verbose=2)
print score[1]

### Save and load keras model

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open(directory+ "/saved_models/equal_dist_keras_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(directory+ "/saved_models/equal_dist_keras_model_weights.h5")
print("Saved model to disk")

### Load model

In [ ]:
# # later...
import pickle
directory = r"deeplearning/data"
# load json and create model
json_file = open(directory+ "/saved_models/equal_dist_keras_model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(directory+ "/saved_models/equal_dist_keras_model_weights.h5")
print("Loaded model from disk")

### Predictions

In [ ]:
OOS_predictions = loaded_model.predict_proba(OOS_vectors)

### SVM

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score ,confusion_matrix
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 

###############################
# df_risk_type = pd.read_excel('/home/hduser/Desktop/risk_category/training_risk_type_country.xls',sep=',')

# df_risk_type = df_risk_type.fillna('')
# df_risk_type['Text'] = df_risk_type['Title'] + " " + df_risk_type['Content'] + " " + df_risk_type['Text']

# df_risk_type = df_risk_type.drop(['Link','Title','Content','Risk_Type'], axis=1)

# print set(df_risk_type["Risk_Subtype"].tolist())

# ###############################
# stop           = set(stopwords.words('english'))
# tf             = TfidfVectorizer(use_idf=True,min_df = 20 ,stop_words=stop).fit(df_risk_type['Text'])

# mails_tfidf    = tf.transform(df_risk_type['Text'])

classifier     = SVC()

parameters     = {'kernel':['rbf'], 'C':[1] ,'gamma' : [0.01]}

clf= GridSearchC
    classifier,  # pipeline from above
    param_grid=parameters,  # parameters to tune via cross validation
    refit=True,  # fit using all data, on the best detected classifier
    n_jobs=4,  # number of cores to use for parallelization; -1 for "all cores"
    scoring='accuracy',  # what score are we optimizing
    cv=StratifiedKFold(train_label, n_folds=2),  # what type of cross validation to use
)

svm_detector = clf.fit(train_vectors,train_label)
print svm_detector.grid_scores_
print ()
print("best score is : "       ,svm_detector.best_score_)
print("best parameters are : " ,svm_detector.best_params_)


In [ ]:
with open(directory+'/saved_models/svm', 'wb') as fp:
    pickle.dump(svm_detector, fp, protocol=2)

### Top Negative Tweets

In [ ]:
df_new_colwise = pd.DataFrame()

In [ ]:
df_new_colwise=pd.concat([new_data,pd.DataFrame(OOS_predictions)],axis=1)
df_new_colwise.sort_values(by=0).head(10000)

In [ ]:
# %matplotlib inline
# df_new_colwise.plot.hist(df_new_colwise[0])

### Top Positive Tweets

In [ ]:
df_new_colwise.sort_values(by=0, ascending = False).head(10000)

In [ ]:
df_new_colwise.loc[df_new_colwise[0]<=0.10, 0] = "negative"
df_new_colwise.loc[((df_new_colwise[0]>0.10) & (df_new_colwise[0]<=0.90)), 0] = "neutral"
df_new_colwise.loc[((df_new_colwise[0]>0.90) & (df_new_colwise[0]<= 1.0)), 0] = "positive"

In [ ]:
df_new_colwise[0].value_counts()

In [ ]:
df_new_colwise["tweet_text_sentiment"].value_counts()

In [ ]:
df_new_colwise.head()

In [ ]:
from nltk.corpus import stopwords
# remove punctuations
def remove_stop_words(tokens):
    tokens = " ".join(tokens)
    tokens = re.sub("[^a-zA-Z]", " ", tokens).split()
    stops = set(stopwords.words("english"))   
    meaningful_words = [w for w in tokens if not w in stops]
    meaningful_words = [w for w in meaningful_words if len(w) > 2]
    return " ".join(meaningful_words)

In [ ]:
for i in df_new_colwise["tokens"]:
    print i
    print remove_stop_words(i)
    break

In [ ]:
df_new_colwise["cleaned_tokens"] = df_new_colwise["tokens"].apply(lambda x: remove_stop_words(x))

In [ ]:
df_new_colwise.head(5)

In [ ]:
# used for getting the Topic Cloud
def collocatedBigrams( total_text, tag = "#amazon"):
    regex_token = RegexpTokenizer(r'\w+')
    
    full_words = []
    for x in total_text:
        full_words.extend(regex_token.tokenize(x))

    bigram_measures = BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(nltk.pos_tag(full_words))
    if tag is not None:
        if len(total_text) < 1000:
            pass
        else:
            finder.apply_freq_filter(min_freq=5)
    bi_final = finder.nbest(bigram_measures.chi_sq, 500)
    imp_bigrams = [[x[0][0], x[1][0]] for x in bi_final if
#                    x[0][1] in ("JJ", "NN", "NNP", "NNS") or x[1][1] in ("NN", "NNP", "NNS", "JJ")
                    len(x[0][0]) >= 3 and len(x[1][0]) >= 3]
    bi_score = finder.score_ngrams(bigram_measures.raw_freq)
    all_bigrams = {" ".join([x[0][0][0], x[0][1][0]]) :x[1] for x in bi_score if len(x[0][0][0]) >= 3 and len(x[0][1][0]) >=3}
    sorted_bigrams = {}
    for bi in imp_bigrams:
        if tag is None and " ".join(bi) in all_bigrams.keys():
            sorted_bigrams[" ".join(bi)] = all_bigrams[" ".join(bi)]
        elif " ".join(bi) in all_bigrams.keys() and tag not in " ".join(bi):
            sorted_bigrams[" ".join(bi)] = all_bigrams[" ".join(bi)]  
    return sorted(sorted_bigrams.items(), key=lambda x: x[1], reverse = True)[:15]  

In [ ]:
collocatedBigrams(list(df_new_colwise[df_new_colwise["tweet_text_sentiment"] == 1]["cleaned_tokens"]))

In [ ]:
collocatedBigrams(list(df_new_colwise[df_new_colwise[0] == "positive"]["cleaned_tokens"]))

In [ ]:
collocatedBigrams(list(df_new_colwise[df_new_colwise["tweet_text_sentiment"] == 0]["cleaned_tokens"]))

In [ ]:
collocatedBigrams(list(df_new_colwise[df_new_colwise[0] == "negative"]["cleaned_tokens"]))

In [ ]:
collocatedBigrams(list(df_new_colwise[df_new_colwise["tweet_text_sentiment"] == 2]["cleaned_tokens"]))

In [ ]:
collocatedBigrams(list(df_new_colwise[df_new_colwise[0] == "neutral"]["cleaned_tokens"]))+

In [ ]:
df_new_colwise[(df_new_colwise[0] == "negative") & (df_new_colwise.tweet_text.str.contains("goddess"))]

In [ ]:
# using the above udf functions
# dat_corr = F.udf(data_correction)
# brand_image = f.udf(brand_imagery, StringType())
# consumer_theme_calc = f.udf(consumer_theme,StringType())

In [ ]:
### Comparison of textblob 

### SVM